Here is some sample code using the YOLOv3 implementation using Python and the OpenCV library for making real time object detection.

the pre-trained weights file for YOLOv3:
https://pjreddie.com/media/files/yolov3.weights

In [1]:
!pip install opencv-python numpy keras

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import cv2
import numpy as np

# Load YOLOv3 model
net = cv2.dnn.readNet("/content/drive/MyDrive/Instant files /yolov3.weights", "/content/drive/MyDrive/Instant files /yolov3.cfg")

# Define the classes
classes = []
with open("/content/drive/MyDrive/Instant files /coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Set the input and output layers
layer_names = net.getLayerNames()
output_layers = [layer_names[i-1] for i in net.getUnconnectedOutLayers()]

# Start the video stream
cap = cv2.VideoCapture(0)

while True:
    # Read the frame
    ret, frame = cap.read()
    if not ret:
        break
    height, width, channels = frame.shape

    # Prepare the input image
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    # Run the forward pass
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Post-processing
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply non-maximum suppression to eliminate redundant overlapping boxes
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Draw the boxes and labels on the frame
    font = cv2.FONT_HERSHEY_PLAIN
    colors = np.random.uniform(0, 255, size=(len(classes), 3))
    if len(indexes) > 0:
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = str(round(confidences[i], 2))
            color = colors[class_ids[i]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label + " " + confidence, (x, y + 20), font, 2, color, 2)

    # Show the frame
    cv2.imshow("Real-time object detection", frame)
    key = cv2.waitKey(1)
    if key == 27:  # Press 'Esc' to exit
        break

# Release the resources
cap.release()
cv2.destroyAllWindows()